In [2]:
import pandas as pd
import pickle
from tqdm import tqdm_notebook

from sklearn.metrics import accuracy_score,roc_auc_score

## Импорт моделей и данных

In [3]:
PATH_TO_DATA="/Users/roman/Inductor/data/megafonDB/"

In [4]:
with open("/Users/roman/Inductor/Roman/xgb_model.pkl","rb") as f:
    xgb=pickle.load(f)

/Users/roman/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [5]:
with open(PATH_TO_DATA+"full_name_MegafonDB_DIRTY.pickle","rb") as f:
    data=pickle.load(f)

## Формирование датасетов

In [6]:
data_3_elements=[i for i in data if len(i.split(" "))==3]

In [7]:
data_2_elements=[i for i in data if len(i.split(" "))==2]

In [8]:
table_3_elements=[j.split(" ") for j in data_3_elements]
table_3_elements=pd.DataFrame(table_3_elements,columns=["Фамилия","Имя","Отчество"])

In [9]:
table_2_elements=[j.split(" ") for j in data_2_elements]
table_2_elements=pd.DataFrame(table_2_elements,columns=["Фамилия","Имя"])

In [10]:
# Формирование датасета 1
SurnameDf=pd.DataFrame(table_3_elements["Фамилия"].values,columns=["word"])
NameDf=pd.DataFrame(table_3_elements["Имя"].values,columns=["word"])
Patronymic=pd.DataFrame(table_3_elements["Отчество"].values,columns=["word"])

SurnameDf["class"]=1
NameDf["class"]=2
Patronymic["class"]=3

data_3_elements=pd.concat([SurnameDf,NameDf,Patronymic])

data_3_elements["word"]=data_3_elements["word"].astype(str)
data_3_elements["lenth"]=data_3_elements["word"].apply(lambda x: len(x)) #Длина слова

data_3_elements=data_3_elements[["word","lenth","class"]]






# Формирование датасета 2
SurnameDf=pd.DataFrame(table_2_elements["Фамилия"].values,columns=["word"])
NameDf=pd.DataFrame(table_2_elements["Имя"].values,columns=["word"])


SurnameDf["class"]=1
NameDf["class"]=2


data_2_elements=pd.concat([SurnameDf,NameDf])

data_2_elements["word"]=data_2_elements["word"].astype(str)
data_2_elements["lenth"]=data_2_elements["word"].apply(lambda x: len(x)) #Длина слова

data_2_elements=data_2_elements[["word","lenth","class"]]

## Построенеи представления данных Bag of letter (верхний и нижний регистр)

In [11]:
def word_to_alphabet(column):
    alphabet=['а','б','в','г','д','е','ё','ж','з','и','й','к','л','м','н','о',
          'п','р','с','т','у','ф','х','ц','ч','ш','щ','ъ','ы','ь','э','ю','я',"e","t","c"] # исключит слово с .
    alphabet=alphabet+[i.upper() for i in alphabet]+["-","."]
    
    alphabet2={i:0 for i in alphabet}
    
    array=[]
    
    for k,word in enumerate(tqdm_notebook(column.values)):
        alphabet3=alphabet2.copy()
        for i in word:
            alphabet3[i]+=1
        array.append(alphabet3.values())
        
    df=pd.DataFrame(array,columns=alphabet)
        
    return df

In [12]:
df_3_elements=word_to_alphabet(data_3_elements["word"])

df_3_elements["lenth"]=data_3_elements["lenth"].values
df_3_elements["class"]=data_3_elements["class"].values

In [13]:
df_2_elements=word_to_alphabet(data_2_elements["word"])

df_2_elements["lenth"]=data_2_elements["lenth"].values
df_2_elements["class"]=data_2_elements["class"].values

### Тестирование на нормальной выборке

In [22]:
predict=xgb.predict(df_3_elements.drop("class",axis=1)[:100000])

In [23]:
accuracy_score(df_3_elements["class"][:100000],predict)

0.97467999999999999

### Тестирование на сложной выборке

In [15]:
predict=xgb.predict(df_2_elements.drop("class",axis=1))

In [16]:
accuracy_score(df_2_elements["class"],predict)

0.64516421702050442